<a href="https://colab.research.google.com/github/SSolanoRuniandes/Notebooks-Aprendizaje-por-Refuerzo-Profundo/blob/main/TareaSemana5.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

![MAIA banner](https://raw.githubusercontent.com/MAIA4361-Aprendizaje-refuerzo-profundo/Notebooks_Tareas/main/Images/Aprendizaje_refuerzo_profundo_Banner_V1.png)

# <h1><center>Tarea Tutorial - Semana 5 <a href="https://colab.research.google.com/github/SSolanoRuniandes/Notebooks-Aprendizaje-por-Refuerzo-Profundo/blob/main/TareaSemana5.ipynb"><img src="https://colab.research.google.com/assets/colab-badge.svg" width="140" align="center"/></a></center></h1>

<center><h1>SAC y A2C</h1></center>

Este tutorial continúa abarcando dos métodos de la familia de algoritmos de gradiente de política: Advantage Actor-Critic (A2C) y Soft Actor-Critic (SAC). Estudiaremos ambos algoritmos usando las implementaciones incluidas dentro de <a href="https://stable-baselines3.readthedocs.io/en/master/index.html">Stable Baselines3</a>, y los usaremos para resolver dos problemas de robótica basados en el entorno de simulación MuJoCo: <a href="https://gymnasium.farama.org/environments/mujoco/hopper/">Hopper</a> y <a href="https://gymnasium.farama.org/environments/mujoco/ant/">Ant</a>. Este notebook tutorial se divide en las siguientes secciones:


# Tabla de Contenidos
1. [Objetivos de Aprendizaje](#scrollTo=Objetivos_de_Aprendizaje)  
2. [Marco Teórico](#scrollTo=Marco_Te_rico)  
3. [Instalación de Librerías](#scrollTo=Instalaci_n_de_Librer_as)  
4. [Familiarización con los Entornos de Gym](#scrollTo=Familiarizaci_n_con_los_Entornos_de_Gym)  
5. [Hopper](#scrollTo=Hopper)  
6. [Ant](#scrollTo=Ant)  
7. [Reflexiones Finales](#scrollTo=Reflexiones_Finales)  
8. [Referencias](#scrollTo=Referencias)

# Objetivos de Aprendizaje  
  
* Comprender las bases teóricas detrás de algoritmos Actor-Crítico.
* Solucionar problemas de robótica basados en simulaciones físicas usando los algoritmos de A2C y SAC.
* Comparar las ventajas y desventajas que existen entre A2C y SAC.


# Marco Teórico  


Recordemos el funcionamiento de los algoritmos de gradiente de política en el aprendizaje por refuerzo. Estos algoritmos, en lugar de realizar una estimación de la función de valor $Q(s,a)$, estiman directamente una política $\pi(s,a,\theta)$ y la mejoran continuamente mediante un proceso de optimización numérica basada en ascenso de gradiente. Uno de estos algoritmos es REINFORCE, estudiado anteriormente en la Tarea de la Semana 4. En este algoritmo, se podía utilizar un <i>baseline</i> para reducir la varianza del entrenamiento. En este escenario, la actualización de REINFORCE está dada por la Ecuación (1):

<center> $\theta_{t+1} \doteq \theta_t + \alpha (G_t-b(S_t)) \frac{\nabla \pi(A_t | S_t, \theta_t)}{\pi(A_t | S_t, \theta_t)}$ &emsp;&emsp;&emsp;$(1)$ </center>

Una selección común para este <i>baseline</i> es un estimativo adicional de la función de valor de estado $V(s)$. Cuando se usa una función de valor como <i>baseline</i>, el término $G_t-b(S_t)$ se transforma en un estimativo de la ventaja (<i>advantage</i>) de la acción $A_t$ en el estado $S_t$. El retorno $G_t$ funciona como un estimador de $Q(S_t,A_t)$, y como el <i>baseline</i> es un estimativo de $V(S_t)$, se define la ventaja formalmente como aparece en la Ecuación (2):

<center> $A(S_t,A_t) = Q(S_t,A_t) - V(S_t)$ &emsp;&emsp;&emsp;$(2)$ </center>

Esta aproximación puede ser vista como una arquitectura actor-crítico, donde la política $\pi$ es el actor que efectúa las acciones, y el <i>baseline</i> es el crítico que evalúa el resultado de dichas acciones. Esta estrategia incorpora un <i>bootstrapping</i> que reduce la varianza al mínimo posible, a costa de introducir un sesgo. La Ecuación (2) sugiere incrementar la probabilidad de las acciones mejor que el promedio, y disminuir la probabilidad de las acciones peores al promedio.

Vale la pena mencionar que el algoritmo Advantage Actor-Critic (A2C) es una versión posterior y sincrónica del algoritmo Asynchronous Advantage Actor-Critic (A3C). En la publicación original [2], A3C es un algoritmo que se formula como una versión asincrónica de un algoritmo actor-crítico, que en lugar de usar <i>experience replay</i>, entrena múltiples agentes en varios hilos de CPU, siendo así una aproximación on-policy más rápida y eficiente computacionalmente. A2C en cambio es la versión síncrona, que no usa agentes en paralelo y puede ser mejor para implementar en GPU. En [2] se muestra que la estimación de la ventaja está dada por la Ecuación (3). En los algoritmos actor-crítico es común utilizar una red neuronal para la estimación de la política y otra red distinta para la estimación de la ventaja, por lo cual se usa la notación $\theta$ para referirse a los parámetros de la red del actor, y $\theta_v$ para la red del crítico.

<center> $A(S_t,A_t, \theta, \theta_v) = \sum_{i=0}^{k-1} \gamma^i R_{t+i} + \gamma^k V(S_{t+k}, \theta_v) - V(s_t, \theta_v)$ &emsp;&emsp;&emsp;$(3)$ </center>

Por otro lado, el algoritmo Soft Actor-Critic (SAC) es un método de aprendizaje por refuerzo que intenta solucionar problemas de otros algoritmos como la complejidad de muestreo, malas propiedades de convergencia, y el requerimiento de un ajuste fino de hiperparámetros. Adicionalmente, SAC es un algoritmo que no busca únicamente maximizar la recompensa, sino que también busca maximizar la entropía. Para comprender mejor este algoritmo, primero se explicará este concepto de entropía y el concepto detrás de <i>Soft Policy Iteration</i>.

Como se mencionó anteriormente, SAC sigue el principio de máxima entropía, lo cual quiere decir que no solo se busca maximizar la recompensa, sino también la entropía de la política. Esto se expresa en la Ecuación (4). En este escenario, $R(S_t,A_t)$, $\mathcal{H}(\pi(\cdot|S_t))$ es la entropía de la política y $\alpha$ es un coeficiente (llamado temperatura) que controla cuánto se valora la entropía (exploración) frente a la recompensa. La entropía en aprendizaje por refuerzo dimensiona cuán impredecible o aleatoria es una política, y una política estocástica con alta entropía explora más. Aumentar la entropía incrementa la exploración, mientras que disminuir la entropía favorece la explotación. La ventaja de esta metodología es que el agente puede alcanzar un comportamiento más robusto que no se estanque en soluciones subóptimas. El proceso de <i>Soft Policy Iteration</i> se refiere al proceso de actualizar la política que regulariza la entropía y también premia la exploración de acciones.  

<center> $ J(\pi) = \sum_{t=0}^{\infty} \mathbb{E}_{(s_t, a_t) \sim \rho_\pi} \left[ r(s_t, a_t) + \alpha \mathcal{H}(\pi(\cdot | s_t)) \right] $ &emsp;&emsp;&emsp;$(4)$ </center>

En la implementación, SAC generalmente utiliza una política Gaussiana para el actor, y para el crítico utiliza Doble Q-Learning, lo cual implica dos redes neuronales adicionales, lo cual puede ser más costoso computacionalmente, pero da mejores resultados al mitigar los sesgos de maximización. SAC se caracteriza por ser un algoritmo <i>off-policy</i> robusto, y es el estado del arte en muchos problemas de robótica, superando en muchos escenarios a otros algoritmos como Proximal Policy Optimization (PPO) y Deep Deterministic Policy Gradient (DDPG).

# Instalación de Librerías  

Corra el siguiente bloque de código para instalar las librerías requeridas en el tutorial. Se encuentra incluído el módulo oficial de Stable Baselines3 que ejecuta A2C y SAC, y los ambientes de Gymnasium que incluyen Hopper y Ant. Esta celda puede tardar un par de minutos en ejecutarse completamente la primera vez.


In [ ]:

#Limpia los registros generados
from IPython.display import clear_output
clear_output()
print("Todas las librerías han sido instaladas correctamente.")

Todas las librerías han sido instaladas correctamente.


# Familiarización con los Entornos de Gym

Los siguientes ambientes de Gymnasium son entornos basados en MuJoCo de Google Deep Mind. MuJoCo (Multi-Joint dynamics with Contact) es un simulador físico utilizado para modelar y estudiar el comportamiento de sistemas dinámicos complejos, muy utilizado en robótica.

## Hopper

El ambiente de <a href="https://gymnasium.farama.org/environments/mujoco/hopper/">Hopper</a> consiste en una pierna en dos dimensiones que tiene como objetivo moverse hacia adelante modificando el torque de sus motores. Concretamente, la pierna se compone de cuatro partes: un torso en la parte superior, un muslo, una pantorrilla y un pie. Y cuenta con tres motores, uno para el muslo ubicado en la cadera, uno en la rodilla, y uno en el tobillo. Un diagrama esquemático del cuerpo y una imagen del entorno de simulación se muestran en la Figura 1.

![Action_space_hopper](https://raw.githubusercontent.com/MAIA4361-Aprendizaje-refuerzo-profundo/Notebooks_Tareas/main/Images/Action_space_hopper.png)

<center>Figura 1. Descripción del ambiente de Hopper. [4]</center>

Como se ve en la Figura 1, el espacio de acciones está descrito por el control de torque en los tres motores ubicados en las bisagras de la pierna. Los índices de los motores son respectivamente 0, 1 y 2, y el rango de control de cada motor se normaliza entre -1 y 1.

Por otro lado, el espacio de observación está descrito por 5 posiciones (<i>qpos</i>) y 6 velocidades (<i>qvel</i>) de las partes individuales del robot.

Finalmente, la recompensa es calculada ponderando 3 aspectos clave: la estabilidad del mecanismo por la cual recibe una recompensa de +1 por cada timestep en el cual la pierna no cae de cierta altura, la posición del torso que se usa para evaluar que la pierna avanza, y una penalización de control que castiga cuando se toman acciones muy grandes en los motores.  

## Ant

El ambiente de <a href="https://gymnasium.farama.org/environments/mujoco/ant/">Ant</a> consiste en un robot cuadrúpedo formado por un torso con forma esférica y cuatro piernas con 2 partes cada una. En total, el robot cuenta con 8 motores, y el objetivo de esta "hormiga" es moverse hacia adelante (dirección derecha de la simulación).

![Action_space_hopper](https://raw.githubusercontent.com/MAIA4361-Aprendizaje-refuerzo-profundo/Notebooks_Tareas/main/Images/Action_space_ant.png)

<center>Figura 2. Descripción del ambiente de Ant. [5]</center>

De forma similar a Hopper, en Ant el espacio de acciones está dado por el control de torque entre -1 y 1 de los 8 motores del sistema dinámico, de acuerdo con los índices de la Figura 2.

En este caso, el espacio de observación está descrito por 13 posiciones (<i>qpos</i>) y 14 velocidades (<i>qvel</i>) de las partes del robot, y 78 elementos adicionales relacionados con el centro de masa y fuerzas externas al sistema.

La recompensa es calculada ponderando 4 aspectos. Al igual que en Hopper, en Ant se le da una recompensa en cada timestep por conservar la estabilidad. También se recompensa que el robot avance en la dirección deseada, y se penalizan acciones muy grandes. El cuarto elemento involucrado en este ambiente corresponde a un castigo si las fuerzas de contacto externas son demasiado grandes.

In [ ]:
#

# =====================================================
# COMPLETAR ===========================================
#

# =====================================================

# Reflexiones Finales

Teniendo en cuenta los resultados observado en ambos ambientes con A2C sin hiperparámetros ajustados, con hiperparámetros ajustados y con SAC, reflexione sobre las siguientes preguntas:


*   ¿Qué algoritmo es más robusto? ¿A qué puede deberse?

*   ¿Qué algoritmo es más costoso computacionalmente? ¿A qué puede deberse?

*   ¿Qué algoritmo obtiene mejores resultados?

*   ¿Resultó útil el ajuste de hiperparámetros?


# Referencias

[1] Sutton, R. S. and Barto, A. G. (2018). Reinforcement Learning: An Introduction. The MIT Press, second edition.

[2] Mnih, V., Puigdomènech, A., Mirza, M., Graves A., Lillicrap T., Harley, T., Silver, D. and Kavukcuoglu, K. (2016). Asynchronous Methods for Deep Reinforcement Learning. cite arXiv:1602.01783.

[3] Haarnoja, T., Zhou, A., Abbeel, P. and Levine, S. (2018). Soft Actor-Critic: Off-Policy Maximum Entropy Deep Reinforcement Learning with a Stochastic Actor. cite arXiv:1801.01290.

[4] Gym Documentation, Hopper. `https://gymnasium.farama.org/environments/mujoco/hopper/`

[5] Gym Documentation, Ant. `https://gymnasium.farama.org/environments/mujoco/ant/`

[6] Stable Baselines3 Documentation, A2C. `https://stable-baselines.readthedocs.io/en/master/modules/a2c.html`

[7] Stable Baselines3 Documentation, SAC. `https://stable-baselines.readthedocs.io/en/master/modules/sac.html`